In [8]:
using Distributions

In [2]:
mu = 3.912115; sigma = 2.71873; 

freq = NegativeBinomial(3920/17 ,28/45)
sev = LogNormal(mu, sigma)

LogNormal{Float64}(μ=3.912115, σ=2.71873)

In [4]:
function  simul_events(n)
    S = zeros(n)
    N = rand(freq, n)
    
    N_pos = findall(N .> 0)
    
    for i in N_pos
        Yi = rand(sev, N[i])
        S[i] = sum(Yi)
    end
    return(S, N)
end

simul_events (generic function with 1 method)

In [ ]:
S = simul_events(10_000)

The next step is to fit the 